In [1]:
!pip install langchain 
!pip install -U langchain-community
!pip install gigachat

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release

In [1]:
import os
import sys

sys.path.append("..")

import time
import pandas as pd
from dotenv import load_dotenv

from typing import Optional

from app.models.llms import GigaChatWrapper
from app.utils.submit import generate_submit
from app.utils.dataset import merge_datasets


load_dotenv()

/home/jovyan/.mlspace/envs/kuzhamuratov-temporalmovqgan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading models...

/home/jovyan/.mlspace/envs/kuzhamuratov-temporalmovqgan/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OK


False

In [2]:
CLIENT_ID = "ce8008fe-77f8-4869-a1ac-2245690df403"
AUTH_KEY = "Y2U4MDA4ZmUtNzdmOC00ODY5LWExYWMtMjI0NTY5MGRmNDAzOmIwODBlZjQ1LWVjMDMtNDJjMy05MjcxLTE5Yjc0YWQ2YWEwYg=="

giga = GigaChatWrapper(
    credentials=AUTH_KEY, 
    model="GigaChat", 
    system_prompt="Ты - профессиональный программист и ментор. Давай очень короткие ответы о синтаксических ошибках в коде. Ошибка в коде гарантированно есть."
)

In [3]:
test = pd.read_excel("../data/test/merged.xlsx")

In [6]:
def predict(row: pd.Series) -> Optional[str]:
    time.sleep(1)
    prefix = f'Описание задачи:\n{row["description"]}\n\nРешение студента:\n{row["student_solution"]}\n\nАвторское решение:\n{row["author_solution"]}'
    return giga.ask(prefix)

In [7]:
i = 1

print("Предикт GPT:")
print(predict(test.iloc[i]))
print("\n\n")
print("Решение студента:")
print(test["student_solution"][i])
print("\n\n")
print("Авторское решение:")
print(test["author_solution"][i])

Предикт GPT:


/home/jovyan/novitskiy/HSE-AI-Assistant-Hack/notebooks/../app/models/llms.py:181: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = self.giga(self.messages)


Ваше решение верное.



Решение студента:
discount  = float(input())
money = int(input())

#ваш код ниже

print(f'Реализация проекта будет стоить {money} тыс. руб. без скидки. Со скидой стоимость составит {money- (money * discount)} тыс. руб.)



Авторское решение:
discount  = float(input())
money = int(input())

#ваш код ниже

print(f'Реализация проекта будет стоить {money} тыс. руб. без скидки. Со скидой стоимость составит {money- (money * discount)} тыс. руб.')


In [10]:
os.makedirs("../processed", exist_ok=True)

In [8]:
generate_submit(
    test_solutions_path="../data/test/merged.xlsx",
    predict_func=predict,
    save_path="../data/processed/gigachat_lite_baseline.csv",
    use_tqdm=True,
)

Predicting: 100%|██████████| 325/325 [17:41<00:00,  3.27s/it]
